In [1]:
%matplotlib widget
%load_ext autoreload
%autoreload 2
from IPython.display import clear_output
import os
import sys

# sys.path.insert(0, os.getenv('HOME')+'/pycode/MscThesis/')
# sys.path.insert(0,r'C:\Users\coren\Documents\PhD\Code\AMFtrack')

import pandas as pd
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    get_current_folders,
    get_folders_by_plate_id,
)


import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.image_processing.extract_graph import (
    from_sparse_to_graph,
    generate_nx_graph,
    sparse_to_doc,
)
from skimage.feature import hessian_matrix_det

# from amftrack.pipeline.functions.image_processing.experiment_class_surf import Experiment
from amftrack.pipeline.launching.run import (
    run_transfer,
)
from amftrack.pipeline.launching.run_super import run_parallel_transfer

import dropbox
from amftrack.util.dbx import upload_folders, download, read_saved_dropbox_state, save_dropbox_state, load_dbx, download, get_dropbox_folders, get_dropbox_video_folders
from subprocess import call
import logging

logging.basicConfig(stream=sys.stdout, level=logging.debug)

/gpfs/home6/svstaalduine/AMF_project/amftrack/util/dbx.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


**To transfer data from dropbox to surfsara**
- select the folders of interest within the folders of dropbox

In [ ]:
directory_targ = "/gpfs/scratch1/shared/amftrackflow/Drp_downs/20221026_Plate452/"

if not os.path.exists(directory_targ):
    os.makedirs(directory_targ)

all_folders_drop, excel_drop = get_dropbox_video_folders("/DATA/FLUORESCENCE/DATA_NileRed/", True)
# folders_drop = all_folders_drop.loc[all_folders_drop["Plate number"] == "452"]

# for xl_adress in excel_drop:
#     file_name = xl_adress.split('/')[-1]
    
#     download(xl_adress, directory_targ+file_name)
clear_output(wait=False)

all_folders_drop

go
2023-05-01 08:48:28,840-[INFO]- dropbox:390 -> Refreshing access token.
2023-05-01 08:48:29,011-[INFO]- dropbox:474 -> Request to files/list_folder
2023-05-01 08:48:30,431-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:48:32,085-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:48:33,954-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:48:35,305-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:48:36,740-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:48:38,233-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:48:39,609-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:48:41,110-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:48:42,554-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:48:44,259-[INFO]- dropbox:474 -> Request to files/list_folder/continue

2023-05-01 08:51:00,837-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:51:02,586-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:51:04,057-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:51:05,673-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:51:08,189-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:51:09,547-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:51:11,144-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:51:12,877-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:51:14,470-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:51:15,932-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:51:17,333-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:51:19,005-[INFO]- dropbox:474 -> Request to files/l

2023-05-01 08:52:54,855-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:52:55,703-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:52:56,646-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:52:57,430-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:52:58,405-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:52:59,150-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:52:59,999-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:53:00,907-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:53:01,741-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:53:02,887-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:53:03,680-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:53:04,451-[INFO]- dropbox:474 -> Request to files/l

2023-05-01 08:54:35,444-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:54:36,388-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:54:37,232-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:54:38,312-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:54:39,218-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:54:40,094-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:54:40,998-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:54:41,863-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:54:45,730-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:54:46,915-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:54:48,207-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:54:49,448-[INFO]- dropbox:474 -> Request to files/l

2023-05-01 08:56:17,929-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:56:19,016-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:56:20,032-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:56:21,180-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:56:22,104-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:56:23,036-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:56:24,129-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:56:25,031-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:56:26,215-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:56:27,123-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:56:28,200-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:56:29,277-[INFO]- dropbox:474 -> Request to files/l

2023-05-01 08:58:00,068-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:58:00,931-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:58:01,885-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:58:02,822-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:58:03,674-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:58:04,963-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:58:05,860-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:58:07,132-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:58:08,077-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:58:10,025-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:58:10,839-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:58:11,709-[INFO]- dropbox:474 -> Request to files/l

2023-05-01 08:59:40,948-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:59:42,137-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:59:43,068-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:59:44,391-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:59:45,273-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:59:46,215-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:59:47,355-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:59:48,202-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:59:49,025-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:59:50,297-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:59:51,453-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 08:59:52,472-[INFO]- dropbox:474 -> Request to files/l

2023-05-01 09:01:28,400-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 09:01:29,335-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 09:01:30,421-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 09:01:31,563-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 09:01:32,384-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 09:01:34,033-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 09:01:35,613-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 09:01:36,486-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 09:01:37,621-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 09:01:38,507-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 09:01:39,730-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 09:01:40,673-[INFO]- dropbox:474 -> Request to files/l

2023-05-01 09:03:14,905-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 09:03:15,837-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 09:03:16,922-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 09:03:18,053-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 09:03:19,409-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 09:03:20,749-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 09:03:21,708-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 09:03:22,673-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 09:03:23,667-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 09:03:24,793-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 09:03:25,750-[INFO]- dropbox:474 -> Request to files/list_folder/continue
2023-05-01 09:03:26,579-[INFO]- dropbox:474 -> Request to files/l

In [2]:

run_parallel_transfer(
    "from_drop.py",
    [directory_targ],
    folders_drop,
    1,
    "10:00:00",
    "transfer_test"
)

# save_dropbox_state("/DATA/FLUORESCENCE/DATA_NileRed/20230201_Plate552/", is_video=True)

go
2023-05-01 08:47:57,305-[INFO]- dropbox:390 -> Refreshing access token.
2023-05-01 08:47:57,570-[INFO]- dropbox:474 -> Request to files/list_folder
2023-05-01 08:47:59,985-[INFO]- dropbox:474 -> Request to files/list_folder/continue


KeyboardInterrupt: 

In [3]:
all_folders_drop

,folder,Plate number,Date Imaged,tot_path_drop,video
0,20230126_Plate528_01,528,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,01
1,20230126_Plate528_02,528,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,02
2,20230126_Plate528_03,528,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,03
3,20230126_Plate528_04,528,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,04
4,20230126_Plate528_05,528,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,05
5,20230126_Plate528_06,528,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,06
6,20230126_Plate528_07,528,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,07
7,20230126_Plate528_08,528,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,08
8,20230126_Plate528_09,528,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,09
9,20230126_Plate528_10,528,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,10


In [4]:
folders_drop = all_folders_drop.loc[all_folders_drop["Plate number"] == "528"]
folders_drop

,folder,Plate number,Date Imaged,tot_path_drop,video
0,20230126_Plate528_01,528,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,01
1,20230126_Plate528_02,528,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,02
2,20230126_Plate528_03,528,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,03
3,20230126_Plate528_04,528,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,04
4,20230126_Plate528_05,528,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,05
5,20230126_Plate528_06,528,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,06
6,20230126_Plate528_07,528,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,07
7,20230126_Plate528_08,528,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,08
8,20230126_Plate528_09,528,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,09
9,20230126_Plate528_10,528,20230126,DATA/FLUORESCENCE/DATA_NileRed/20230126_Plate5...,10


In [5]:
directory_targ = "/gpfs/scratch1/shared/amftrackflow/Drp_downs/20230201_Plate528/"

In [7]:
for xl_adress in excel_drop:
    file_name = xl_adress.split('/')[-1]
    
    download(xl_adress, directory_targ+file_name)

2023-04-11 11:22:49,196-[INFO]- dropbox:390 -> Refreshing access token.
2023-04-11 11:22:49,395-[INFO]- dropbox:474 -> Request to files/download


In [8]:
run_parallel_transfer(
    "from_drop.py",
    [directory_targ],
    folders_drop,
    1,
    "10:00:00",
    "transfer_test"
)
clear_output(wait=False)